In [ ]:
import numpy as np

import tweeterator
from loader import Loader
from data_generator import DataGenerator

Setup training parameters

In [ ]:
input = 'data/vicinitas_user_tweets.xlsx'
net_type = 'RNN'
latent_dim = 50
window = 5
dropout = 0
batch_size = 32
epochs = 100
learning_rate = 0.0001
perc_test = 0.2
n_hidden_layers = 2
regex_to_remove = ['^#.+:\s+']

Train and get the trained model and the word dictionaries

In [ ]:
model, w2i, i2w = tweeterator.train(input, net_type, latent_dim, window, dropout, batch_size, epochs, learning_rate,
                          perc_test, n_hidden_layers, regex_to_remove)

Test starting from the first words of the first training phrase

In [ ]:
loader = Loader()
data = loader.load(input, window=window + 1, regex_to_remove=regex_to_remove)
generator = DataGenerator(data, w2i, window, batch_size)
test_example = next(generator)[0]

output_size = 40
output_int = np.empty(output_size, dtype=int)

# Set the first window elements to the start of the phrase
output_int[:window] = test_example[0]

# Predict the next word from the preceding ones (using the predicted words)
for i in range(0, output_int.size - window):
    input_int = output_int[np.newaxis, i:window + i, np.newaxis]
    prediction = model(input_int)
    word_int = np.argmax(prediction)
    output_int[window + i] = word_int

# Convert integers to words
output = []
for i in range(len(output_int)):
    word_int = output_int[i]
    word = i2w[word_int]
    output.append(word)

Visualise the produced output

In [ ]:
' '.join(output)